In [19]:
import os
import pandas as pd
import re

In [2]:
os.chdir("..")
data_path="data/interim"


In [3]:
pwd

'/Users/federicocirett/Documents/GitHub/ingCaracteristicasProyecto_'

Now that we are in the preferable working directory, I'll explore each data file's variables in order to figure out how to join each dataframe into a big dataset. Let's begin chronologically:

# 1950-1995

# 2000-2020

In [5]:
df_2000=pd.read_csv(f"{data_path}/2000.csv")
df_2005=pd.read_csv(f"{data_path}/2005.csv")
df_2010=pd.read_csv(f"{data_path}/2010.csv")
df_2020=pd.read_csv(f"{data_path}/2020.csv")

/var/folders/m5/26l5nfs54r94_664jm4k6zvr0000gn/T/ipykernel_9866/36002156.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020=pd.read_csv(f"{data_path}/2020.csv")


In [15]:
df_2020.columns[:60]

Index(['Unnamed: 0', 'Clave de entidad federativa', 'Entidad federativa',
       'Clave de municipio o demarcación territorial',
       'Municipio o demarcación territorial', 'Clave de localidad',
       'Localidad', 'Longitud', 'Latitud', 'Altitud', 'Población total',
       'Población femenina', 'Población masculina', 'Población de 0 a 2 años',
       'Población femenina de 0 a 2 años', 'Población masculina de 0 a 2 años',
       'Población de 3 años y más', 'Población femenina de 3 años y más',
       'Población masculina de 3 años y más', 'Población de 5 años y más',
       'Población femenina de 5 años y más',
       'Población masculina de 5 años y más', 'Población de 12 años y más',
       'Población femenina de 12 años y más',
       'Población masculina de 12 años y más', 'Población de 15 años y más',
       'Población femenina de 15 años y más',
       'Población masculina de 15 años y más', 'Población de 18 años y más',
       'Población femenina de 18 años y más',
       'P

I think it will be a good idea to split the population columns into categories. For example, the 'Población de 15 años y más' column will be abreviated as 'POB_15+', and each description will be abreviated as well.

I'll begin with abreviating the first part of the population columns.









In [16]:
columns={'Clave de entidad federativa':'CVE_ENT',
         'Entidad federativa':'ENT',
         'Clave de municipio o demarcación territorial':'CVE TERR',
         'Municipio o demarcación territorial':'MUN',
         'Clave de localidad':'CVE_LOC',
         'Localidad':'LOCALIDAD',
         'Longitud':'LON',
         'Latitud':'LAT',
         'Altitud':'ALTI',
         'Población total':'POB_TOT'
}



Now, I need to find a good use of regular expressions in order to speed things up. I want for the next columns to have the name structure: "POB_[age_range]_[category]". 

In [22]:
age_range_pattern=r'\d+ a \d+' 
text='Población masculina de 8 a 14 años que no sabe leer y escribir'
match=re.search(age_pattern,text)
match.group()

'8 a 14'

In [55]:
def age_range(text):
    # Case for specified age range:
    age_range_pattern=r'\d+ a \d+' 
    match=re.search(age_range_pattern,text)
    if match:
        return match.group()
    # Case for ambiguos age range:
    ambiguos_age_range_pattern=r"\d+ años y más"
    match=re.search(ambiguos_age_range_pattern,text)
    if match:
        text=match.group(0)
        age_range=f"{re.findall(r'\d+',text)[0]}+"
        return age_range



In [57]:
text='Población masculina de 8 a 14 años que no sabe leer y escribir'
age_range(text)

'8 a 14'

In [58]:
text="Población masculina de 12 años y más ocupada"
age_range(text)

'12+'

In [60]:
df_2020.columns[100:]

Index(['Población femenina de 70 a 74 años',
       'Población masculina de 70 a 74 años', 'Población de 75 a 79 años',
       'Población con discapacidad',
       'Población con discapacidad para hablar o comunicarse',
       'Población con discapacidad para oír, aun usando aparato auditivo',
       'Población con discapacidad para vestirse, bañarse o comer',
       'Población con discapacidad para recordar o concentrarse',
       'Población con limitación',
       'Población con limitación para caminar, subir o bajar',
       'Población con limitación para ver, aun usando lentes',
       'Población con limitación para hablar o comunicarse',
       'Población con limitación para oír, aun usando aparato auditivo',
       'Población con limitación para vestirse, bañarse o comer',
       'Población con limitación para recordar o concentrarse',
       'Población con algún problema o condición mental',
       'Población sin discapacidad, limitación, problema o condición mental',
       'Po

In [61]:
def category(text):
    categories={
        "que no asiste a la escuela":"NAE",
        "que asiste a la escuela":"AE",
        "que no sabe leer y escribir":"NSLE",
        "analfabeta":"ANBT",
        "sin escolaridad":"SIN_E",
        "primaria completa":"PRIM_COM",
        "primaria incompleta":"PRIM_INC",
        "secundaria completa":"SEC_COM",
        "secundaria incompleta":"SEC_INC",
        "educación posbásica":"POSBASICA"
    }
    for replacement,category in categories.items():
        match=re.search(fr'{replacement}',text)
        if match:
            return category
    print("No matches :( ")

In [62]:
text='Población masculina de 15 años y más con primaria incompleta'
category(text)

'PRIM_INC'

In [65]:
def population(text):
    match=re.search("Población",text)
    if match:
        sex=re.search("femenina|masculina",text)
        if sex:
            letter=str.upper(sex.group()[0])
            return f"POB {letter}"
        return "POB"

In [67]:
text='Población de 15 años y más con primaria incompleta'
population(text)

'POB'